In [8]:
#nltk library used to import list of words. However, some aren't in dictionary?
from nltk.corpus import words
#pandas used to create and manipulate dataframe. Could possibly get away with just dictionaries.
import pandas as pd

def load(username="Tom", password="Tom"):
    """Function that instanciates required variables for the play function. Could also be adapted to allow usernames and passwords."""
    
    if username in ["Tom"] and password in ["Tom"]:

        #List of 5 letter words that can be pulled from the nltk library.
        #A lot of strange words within this. Can a better library be found?
        global five_let_list
        five_let_list = list(set([i.lower() for i in words.words() if len(i) == 5]))

        #Position dictionary holds the letters which are in the correct position.
        global corr_pos_dict
        corr_pos_dict = {'pos_0': "", 'pos_1': "", 'pos_2': "", 'pos_3': "", 'pos_4': ""}

        #Wrong list hold the letters that do not occur anywhere in the word.
        global wrong_let_list
        wrong_let_list = []

        #Letter dictionary holds the letters which are correct but the position they do not occur in.
        global corr_let_dict
        corr_let_dict = {'pos_0': [], 'pos_1': [], 'pos_2': [], 'pos_3': [], 'pos_4': []}

        #Letters contained list which is an unordered list of the confirmed letters within the word.
        global let_contained_list
        let_contained_list = []

#Run load at this point otherwise the remaining functions show an error due to uninstanciated parameter defaults.
load()

def contains_all(words, letters):
    """Function to check what words within a list contain all of the letters within another list."""

    for word in list(words.copy()):

        for letter in letters:
            if word.__contains__(letter): continue

            else:
                try: words.remove(word)
                except: pass

    return words

def play(corr_pos_dict = corr_pos_dict, wrong_let_list = wrong_let_list, corr_let_dict = corr_let_dict, five_let_list = five_let_list):
    """Function that allows user to interact with the script."""

    #Instanciate a dataframe that holds each remaining five letter word, each on an individual row.
    #Each column holds a letter of the word in the position it occurs.
    df = pd.DataFrame(data=[list(i) for i in five_let_list], columns=[i for i in corr_pos_dict.keys()])
    #Add a columns to hold a score that will be recalculated after each round of guessing.
    df['score'] = 0



    #This is where the score for each word is calculated. Begin by iterating through each remaining word and...
    #assign the word one point each time there is another remaining word with the same letter in the same position.
    #Even if the  highest scoring word is wrong, it should rule out the maximum number of alternatives and give the most information.
    #Research GAME THEORY...
    for i in df.index:
        for pos in corr_pos_dict.keys():
            letter = df.loc[i, pos]
            df.loc[i, 'score'] += df[pos].value_counts().to_frame().loc[letter, pos]

    #Ensure the highest scoring words are at the tope of the dataframe and present the higest scoring words to the user.
    df = df.copy().sort_values(by='score', ascending=False)
    print(df.head())
    print("")

    #Ask the user word they would like to guess and present it back to them.
    guess_word = input("What word did you guess?").lower()
    print(guess_word)

    #After user makes a guess, they must tell the script which letters are correct and which are in the correct position.
    #This part could definitely be made a bit more user friendly using try, except.
    #Currently, users cannot make any mistake without having to start again.
    for i in range(5):

        #If poisition is known, this prevents the user from being asked again about that position.
        if len(corr_pos_dict[f'pos_{i}']) > 0: continue

        if input(f"Was letter {i+1} a correct letter? (y/n)").lower()[0] == "y":
            let_contained_list.append(guess_word[i])

            if input(f"Was letter {i+1}  in the correct position? (y/n)").lower()[0] == "y":
                corr_pos_dict[f'pos_{i}'] = guess_word[i]

            else:
                corr_let_dict[f'pos_{i}'].append(guess_word[i])
                
        else:
            wrong_let_list.append(guess_word[i])

    #This part removes all words that do mot have specific letters in the known positions.
    for i in range(5):

        if len(corr_pos_dict[f'pos_{i}']) == 1:

            for word in list(five_let_list.copy()):

                if word[i] != corr_pos_dict[f'pos_{i}']:
                    try: five_let_list.remove(word)
                    except: pass

    #This part removes all words that contain a letter that is known to be wrong.        
    for j in range(5):
        
        for word in list(five_let_list.copy()):

            if word[j] in wrong_let_list:
                try: five_let_list.remove(word)
                except: pass

    #This part removes words that have known letters in the incorrect position.
    for i in range(5):
        
        if len(corr_let_dict[f'pos_{i}']) > 0:

            for word in list(five_let_list.copy()):

                if word[i] in corr_let_dict[f'pos_{i}']:
                    try: five_let_list.remove(word)
                    except: pass

    #Final check that all remaining words contain all of the known letters regardless of position.
    five_let_list = contains_all(five_let_list.copy(), let_contained_list)

    #Display variables that are being built throughout the process.
    print(corr_let_dict)
    print(corr_pos_dict)
    print(wrong_let_list)
    print(five_let_list[0:10])

    #If there are more than 5 words left, continue playing.
    if len(five_let_list) > 5:
        play(corr_pos_dict = corr_pos_dict, wrong_let_list = wrong_let_list, corr_let_dict = corr_let_dict, five_let_list = five_let_list)
    #If there are less than 5 words remaining there is no point in continuing to play the game as they are all present within df head.
    else:
        print(pd.DataFrame(data=[list(i) for i in five_let_list], columns=[i for i in corr_pos_dict.keys()]).head())
        print("\nThese are the final words remaining. Choose wisely!")

#Run play function.
play()

     pos_0 pos_1 pos_2 pos_3 pos_4  score
471      s     o     r     e     e   6263
1187     s     a     l     a     y   6062
9687     s     a     c     a     e   5822
7221     s     a     r     a     n   5802
6345     s     a     i     t     e   5773

sweat
{'pos_0': [], 'pos_1': [], 'pos_2': [], 'pos_3': [], 'pos_4': []}
{'pos_0': 's', 'pos_1': '', 'pos_2': 'e', 'pos_3': '', 'pos_4': ''}
['w', 'a', 't']
['skeed', 'siege', 'seege', 'sheng', 'smeek', 'speen', 'sheer', 'skelp', 'sueve', 'specs']
   pos_0 pos_1 pos_2 pos_3 pos_4  score
23     s     k     e     e     l    161
16     s     p     e     e     l    160
0      s     k     e     e     d    159
24     s     k     e     e     r    159
20     s     p     e     e     r    158

skeel
{'pos_0': [], 'pos_1': ['k'], 'pos_2': [], 'pos_3': [], 'pos_4': ['l']}
{'pos_0': 's', 'pos_1': '', 'pos_2': 'e', 'pos_3': 'e', 'pos_4': ''}
['w', 'a', 't']
['sleek']
  pos_0 pos_1 pos_2 pos_3 pos_4  score
0     s     l     e     e     k      5




IndexError: string index out of range